In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE102807"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE102807"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE102807.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE102807.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE102807.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"ARID2 promotes clear cell renal cell carcinoma in the absence of functional PBRM1"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: MEF'], 1: ['adenovirus: GFP', 'adenovirus: Cre'], 2: ['genotype/variation: control', 'genotype/variation: loss of Pbrm1 expression']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a MEF (mouse embryonic fibroblast) cell line study
# with adenovirus treatment and genotype variations. This suggests it might be a gene expression dataset,
# but there's no clear indication of human gene expression data as required.
is_gene_available = False  # No clear indication of human gene expression data

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics, we don't see clear trait (Kidney Clear Cell Carcinoma), age, or gender information
# The data appears to be from a cell line experiment rather than human samples

# 2.1 Data Availability
trait_row = None  # No trait information found
age_row = None    # No age information found
gender_row = None # No gender information found

# 2.2 Data Type Conversion Functions
# Define these functions even though we don't have the data, to maintain code structure
def convert_trait(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # For kidney clear cell carcinoma, binary classification (0 for normal, 1 for cancer)
    if value.lower() in ['normal', 'control', 'healthy']:
        return 0
    elif 'carcinoma' in value.lower() or 'cancer' in value.lower() or 'tumor' in value.lower():
        return 1
    return None

def convert_age(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        # Convert to float to handle both integers and decimal ages
        return float(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Convert to binary (0 for female, 1 for male)
    if value.lower() in ['female', 'f']:
        return 0
    elif value.lower() in ['male', 'm']:
        return 1
    return None

# 3. Save Metadata
# Initial filtering check: both gene and trait data should be available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip clinical feature extraction


A new JSON file was created at: ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"ARID2 promotes clear cell renal cell carcinoma in the absence of functional PBRM1"
Line 1: !Series_geo_accession	"GSE102807"
Line 2: !Series_status	"Public on Aug 14 2018"
Line 3: !Series_submission_date	"Aug 18 2017"
Line 4: !Series_last_update_date	"Mar 08 2022"
Line 5: !Series_summary	"This SuperSeries is composed of the SubSeries listed below."
Line 6: !Series_overall_design	"Refer to individual Series"
Line 7: !Series_type	"Expression profiling by array"
Line 8: !Series_type	"Genome binding/occupancy profiling by high throughput sequencing"
Line 9: !Series_type	"Expression profiling by high throughput sequencing"
Found table marker at line 73
First few lines after marker:
"ID_REF"	"GSM2742483"	"GSM2742484"	"GSM2742485"	"GSM2742486"	"GSM2742487"	"GSM2742488"
17210855	10.07882196	9.892336836	10.0362495	10.41165178	10.26672026	10.29559358
17210869	10.25753481	10.20831352	10.25449864	10.15972146	10.34576857	10.26063389
17210887

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the gene data
# The identifiers appear to be numeric IDs (17210855, 17210869, etc.)
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# They appear to be probe IDs from a microarray platform
# These would need to be mapped to human gene symbols for interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 204029 rows

Gene annotation preview (first few rows):
{'ID': ['17210850', '17210852', '17210855', '17210869', '17210883'], 'probeset_id': ['17210850', '17210852', '17210855', '17210869', '17210883'], 'SPOT_ID': ['chr1(+):3102016-3102125', 'chr1(+):3466587-3513553', 'chr1(+):4807823-4846739', 'chr1(+):4857694-4897909', 'chr1(+):4970857-4976820'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': [3102016.0, 3466587.0, 4807823.0, 4857694.0, 4970857.0], 'stop': [3102125.0, 3513553.0, 4846739.0, 4897909.0, 4976820.0], 'total_probes': [8.0, 23.0, 20.0, 21.0, 23.0], 'gene_assignment': ['---', '---', 'NM_008866 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000027036 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC013536 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC052848 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// U89352 // Lypla1 // lysophos

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From examining the gene_data and gene_annotation dataframes:
# - 'ID' in gene_annotation contains the same identifiers as the index in gene_data
# - 'gene_assignment' contains gene symbol information, but needs parsing

# Check the first few gene_assignment values to understand the format
print("Examining gene_assignment format in the first few rows:")
print(gene_annotation['gene_assignment'].head(3).tolist())

# Create a modified mapping function that works with this specific gene annotation format
def get_mouse_gene_mapping(annotation_df):
    """
    Extract probe ID to gene symbol mapping from mouse gene annotation data.
    The gene_assignment column has a complex format with multiple gene entries separated by /// 
    and each entry having gene identifier information and gene symbol.
    """
    mapping_data = annotation_df[['ID', 'gene_assignment']].copy()
    mapping_data = mapping_data.rename(columns={'gene_assignment': 'Gene'})
    
    # Extract mouse gene symbols from the gene_assignment text
    def extract_mouse_gene_symbols(text):
        if text == '---' or pd.isna(text):
            return []
        
        # Extract gene symbols following the pattern RefSeq_ID // GeneSymbol // Description
        genes = []
        # Look for patterns like "// Lypla1 //" where Lypla1 is the gene symbol
        segments = text.split('///')
        for segment in segments:
            # Find the pattern "// GeneSymbol //"
            match = re.search(r'//\s+([A-Za-z0-9-]+)\s+//', segment)
            if match:
                genes.append(match.group(1))
        
        # Remove duplicates while preserving order
        return list(dict.fromkeys(genes))
    
    mapping_data['Gene'] = mapping_data['Gene'].apply(extract_mouse_gene_symbols)
    
    # Filter rows with no gene symbols
    mapping_data = mapping_data[mapping_data['Gene'].apply(len) > 0]
    
    return mapping_data

# 2. Get gene mapping dataframe
gene_mapping = get_mouse_gene_mapping(gene_annotation)

# Print the mapping dataframe to verify
print("\nGene mapping preview (first few rows):")
print(gene_mapping.head())
print(f"Number of probes with gene mappings: {len(gene_mapping)}")

# Check if the IDs in gene_mapping exist in gene_data
common_ids = set(gene_mapping['ID']).intersection(set(gene_data.index))
print(f"Number of common IDs between gene_mapping and gene_data: {len(common_ids)}")

# 3. Apply gene mapping to convert probe values to gene expression data
# Since we're working with mouse data but our project is about human disease,
# we'll map the mouse genes but mark the data as not applicable for human studies
try:
    # Convert gene_data index to string type to match mapping
    gene_data.index = gene_data.index.astype(str)
    
    # Apply gene mapping
    gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)
    
    # Print results
    print("\nMapped gene expression data preview (first few rows):")
    print(gene_data_mapped.head())
    print(f"\nGene expression data dimensions: {gene_data_mapped.shape}")
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_mapped.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
    # This is mouse data not human data, so we should mark it as not available
    is_gene_available = False
    print("Note: Although we mapped the gene expression, this is mouse data not human data.")
    print("For human trait associations, we'll mark gene expression as unavailable.")
    
except Exception as e:
    print(f"Error applying gene mapping: {e}")
    is_gene_available = False


Examining gene_assignment format in the first few rows:
['---', '---', 'NM_008866 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000027036 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC013536 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC052848 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// U89352 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// CT010201 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000134384 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000150971 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000134384 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000155020 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000141278 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// AK050549 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// AK167231 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000115529 // Ly


Gene mapping preview (first few rows):
         ID       Gene
2  17210855   [Lypla1]
3  17210869    [Tcea1]
5  17210887  [Atp6v1h]
6  17210904    [Oprk1]
7  17210912   [Rb1cc1]
Number of probes with gene mappings: 27017
Number of common IDs between gene_mapping and gene_data: 27017

Mapped gene expression data preview (first few rows):
Empty DataFrame
Columns: [GSM2742483, GSM2742484, GSM2742485, GSM2742486, GSM2742487, GSM2742488]
Index: []

Gene expression data dimensions: (0, 6)
Gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE102807.csv
Note: Although we mapped the gene expression, this is mouse data not human data.
For human trait associations, we'll mark gene expression as unavailable.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# Use is_trait_available from previous steps, don't recheck trait_row
print("\nChecking trait availability from previous analysis...")
# Load the clinical data from the previously saved file
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    # Rename the column '1' to the trait name if it exists
    if '1' in clinical_df.columns:
        clinical_df = clinical_df.rename(columns={'1': trait})
        print(f"Renamed column '1' to '{trait}'")
    is_trait_available = trait in clinical_df.columns
except Exception as e:
    print(f"Error loading clinical data: {e}")
    clinical_df = pd.DataFrame()
    is_trait_available = False

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not normalized_gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("Clinical columns:", list(clinical_df.columns))
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Transpose clinical data so samples are rows
        if clinical_df.shape[0] == 1:  # If it's currently 1 row with samples as columns
            clinical_df = clinical_df.T
            clinical_df.columns = [trait]
            print(f"Transposed clinical data to shape: {clinical_df.shape}")
        
        # Now link clinical and genetic data
        linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 3. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples after missing value handling
                if linked_data.shape[0] > 0:
                    # 4. Determine whether the trait and demographic features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("Error: All samples were removed during missing value handling.")
                    is_biased = True
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        if not is_trait_available:
            print(f"Reason: Trait column '{trait}' is not available in clinical data.")
        if normalized_gene_data.empty:
            print("Reason: Gene expression data is empty.")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' in locals() and linked_data.empty:
    if not normalized_gene_data.empty and not clinical_df.empty:
        note = "Dataset failed in linking phase. Clinical data structure could not be properly matched with gene expression data."
    elif normalized_gene_data.empty:
        note = "Dataset failed in gene mapping phase. Could not properly map probe IDs to gene symbols."
    elif clinical_df.empty:
        note = "Dataset failed in clinical data extraction. No valid trait information available."
elif 'is_biased' in locals() and is_biased:
    note = "Dataset passed initial processing but contains severely biased trait distribution."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=not normalized_gene_data.empty,  # Set based on actual gene data availability
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 0 unique gene symbols
Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE102807.csv

Checking trait availability from previous analysis...
Error loading clinical data: [Errno 2] No such file or directory: '../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE102807.csv'

Linking clinical and genetic data...
Cannot link data: clinical or genetic data is missing.
Reason: Trait column 'Kidney_Clear_Cell_Carcinoma' is not available in clinical data.
Reason: Gene expression data is empty.

Performing final validation...
Abnormality detected in the cohort: GSE102807. Preprocessing failed.
Dataset not usable for Kidney_Clear_Cell_Carcinoma association studies. Data not saved.
